In [3]:
import os 
import time
import pandas as pd


import keras 
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from keras import layers
from keras.layers import Conv2DTranspose

from tensorflow.keras import backend as K
from tensorflow.keras.layers import TimeDistributed
import numpy as np
import librosa
import matplotlib.pyplot as plt
import librosa.display

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score


# Detection semi-supérvisé des signaux d'usure des roulements

## Import des données

In [ ]:
train_data_2 = pd.read_csv("Dataset5/train.csv")
test_data_2 = pd.read_csv("Dataset5/test.csv")

## Creation de modèle ( CNN AE ) 

In [ ]:
class ConvAutoencoder:
    @staticmethod
    def build(width, height, depth, filters=(32, 64), latentDim=100):
        # initialize the input shape to be "channels last" along with
        # the channels dimension itself
        # channels dimension itself
        inputShape = (height, width, depth)
        chanDim = -1
        # define the input to the encoder
        inputs = Input(shape=inputShape)
        x = inputs
        # loop over the number of filters
        for f in filters:
        # apply a CONV => RELU => BN operation
            x = Conv2D(f, (3, 3), strides=2, padding="same")(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization(axis=chanDim)(x)
        # flatten the network and then construct our latent vector
        volumeSize = K.int_shape(x)
        x = Flatten()(x)
        latent = Dense(latentDim)(x)
        # build the encoder model
        encoder = Model(inputs, latent, name="encoder")
        # start building the decoder model which will accept the
        # output of the encoder as its inputs
        latentInputs = Input(shape=(latentDim,))
        x = Dense(np.prod(volumeSize[1:]))(latentInputs)
        x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)
        # loop over our number of filters again, but this time in
        # reverse order
        for f in filters[::-1]:
            # apply a CONV_TRANSPOSE => RELU => BN operation
            x = Conv2DTranspose(f, (3, 3), strides=2,
                padding="same")(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization(axis=chanDim)(x)
        # apply a single CONV_TRANSPOSE layer used to recover the
        # original depth of the image
        x = Conv2DTranspose(depth, (3, 3), padding="same")(x)
        outputs = Activation("sigmoid")(x)
        # build the decoder model
        decoder = Model(latentInputs, outputs, name="decoder")
        # our autoencoder is the encoder + decoder
        autoencoder = Model(inputs, decoder(encoder(inputs)),
        name="autoencoder")
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
        autoencoder.compile(loss="mse", optimizer=optimizer)
        return autoencoder

## Preparation des données

In [ ]:

def build_spectrograms(data,window,saving_folder,step,label_col = 'Class'):
    i = 0
    range_df = i*step
    while range_df+window < len(data):
        range_df = i*step
        y = data[range_df:range_df+window].value.values
        img_class = 1 if data[range_df:range_df+window][label_col].sum() > 0.3*window else  0
        S_left = librosa.stft(y, center=True, n_fft=int(window/2))
        fig, ax = plt.subplots()
        img = librosa.display.specshow(librosa.amplitude_to_db(S_left,
                                                               ref=np.max),
                                       y_axis='log', x_axis='time', ax=ax)
        ax.set_title('Power spectrogram')
        fig.colorbar(img, ax=ax, format="%+2.0f dB")
        plt.savefig(f"{saving_folder}/class_{img_class}/{range_df}_{range_df+window-1}.png")
        print(f"{saving_folder}/class_{img_class}/{range_df}_{range_df+window-1}.png")
        i+=1


In [ ]:
# build_spectrograms(train_data_2,500,"Spectrograms6/Train",200)
# build_spectrograms(test_data_2,500,"Spectrograms6/Test",200)

## Chargement des spectrogrammes

In [ ]:
#chargement des données de spéctrogramme d'entrainement
train_folder = "SpectrogramsNB2/Train"
train_images = []
ano_images = [] 
class_data = ["class_0","class_1"]
for label in class_data:
    for i,img in enumerate(os.listdir(f"{train_folder}/{label}/")):
        if 'png' in img:
            image = tf.keras.utils.load_img(f"{train_folder}/{label}/{img}",color_mode = "grayscale",target_size=(100, 100))
            input_arr = tf.keras.utils.img_to_array(image)/255.0
            if label == "class_0":    
                train_images.append(input_arr)
            else:
                ano_images.append(input_arr)


In [ ]:
X_train0_CAE = np.array(train_images) # spectrograms de données normal train
X_train1_CAE = np.array(ano_images) # spectrograms de données anormal train

In [ ]:
#chargement des données de spéctrogramme de test
test_folder = "SpectrogramsNB2/Test"
test_images0 = []
test_images1 = [] 
class_data = ["class_0","class_1"]
for label in class_data:
    for i,img in enumerate(os.listdir(f"{test_folder}/{label}/")):
        if 'png' in img:
            image = tf.keras.utils.load_img(f"{test_folder}/{label}/{img}",color_mode = "grayscale",target_size=(100, 100))
            input_arr = tf.keras.utils.img_to_array(image)/255.0
            if label == "class_0":    
                test_images0.append(input_arr)
            else:
                test_images1.append(input_arr)

In [ ]:
X_test0CAE = np.array(test_images0) # spectrograms de données normal test
X_test1CAE = np.array(test_images1) # spectrograms de données anormal test

## Apprentissage du modèle

In [ ]:
CAE = ConvAutoencoder.build(100, 100, 1) # construction de l'architecture

In [ ]:
start = time.time()
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
CAE.fit(X_train0_CAE,X_train0_CAE, epochs=100, verbose = True, callbacks= [callback])
print(f"model trained in {(time.time()-start)/60} minutes")

## Prediction

In [ ]:
predictions0CAE = CAE.predict(X_train0_CAE) # normal data predictions
predictions1CAE = CAE.predict(X_train1_CAE) # anomalies predictions

In [ ]:
r_e_CAE = np.sum(np.sum((X_train0_CAE-predictions0CAE) ** 2, axis=1),axis = 1) # computing reconstruction error of normal train data 
r_e_anos_CAE = np.sum(np.sum((X_train1_CAE-predictions1CAE) ** 2, axis=1),axis = 1) # computing reconstruction error of abnormal train data 
threshold_t_CAE = np.quantile(r_e_CAE,0.95) # setting reconstruction error threshold

In [ ]:
prediction_test0CAE = CAE.predict(X_test0CAE)
prediction_test1CAE = CAE.predict(X_test1CAE)

In [ ]:
r_e_test0CAE = np.sum(np.sum((X_test0CAE-prediction_test0CAE) ** 2, axis=1),axis=1)
r_e_test1CAE = np.sum(np.sum((X_test1CAE-prediction_test1CAE) ** 2, axis=1),axis=1)

In [ ]:
decision0CAE = -r_e_test0CAE + threshold_t_CAE
decision1CAE = -r_e_test1CAE + threshold_t_CAE
df0CAE = pd.DataFrame(decision0CAE, columns=['v'])
df1CAE = pd.DataFrame(decision1CAE, columns=['v'])

In [ ]:
# computing f1 score of minority class
recall = len(df1CAE[df1CAE.v<0])/len(df1CAE)
precision = len(df1CAE[df1CAE.v<0])/(len(df1CAE[df1CAE.v<0])+len(df0CAE[df0CAE.v<0]))
f1 = 2*precision*recall/(precision+recall)

In [ ]:
# computing f1 score of majority class
recall_maj = len(df0CAE[df0CAE.v>0])/len(df0CAE)
precision_maj = len(df0CAE[df0CAE.v>0])/(len(df1CAE[df1CAE.v>0])+len(df0CAE[df0CAE.v>0]))
f1_maj = 2*precision_maj*recall_maj/(precision_maj+recall_maj)

### Test score

In [ ]:
print(f"Test scores:")
print(f"minority class F1 score {f1}")
print(f"majority class F1 score {f1_maj}")
print(f"average F1 score {(f1+f1_maj)/2}")